In [2]:
import re
import json
import nbformat
import google.generativeai as genai

# Cấu hình API Key
genai.configure(api_key="")  

class LessonDSLProcessor:
    def __init__(self, dsl_text):
        self.dsl_text = dsl_text
        self.lesson = {"title": "", "sections": []}

    def parse(self):
        lesson_match = re.search(r'lesson\s+"(.+?)"\s+{', self.dsl_text)
        if lesson_match:
            self.lesson["title"] = lesson_match.group(1)

        section_matches = re.finditer(r'section\s+"(.+?)"\s+{(.*?)}', self.dsl_text, re.DOTALL)
        for match in section_matches:
            title, body = match.groups()
            content_match = re.search(r'content\s+"""(.*?)"""', body, re.DOTALL)
            generate_match = re.search(r'generate\s+"(.+?)"(?:\s*\{parameters\s*{(.*?)}})?', body, re.DOTALL)

            section = {"title": title, "content": ""}
            if content_match:
                section["content"] = content_match.group(1).strip()
            elif generate_match:
                prompt = generate_match.group(1)
                params = self.extract_parameters(generate_match.group(2))
                section["content"] = self.generate_with_llm(prompt, params)

            self.lesson["sections"].append(section)

    def extract_parameters(self, param_text):
        """ Trích xuất tham số từ phần parameters { key: value } """
        if not param_text:
            return {}
        params = {}
        matches = re.findall(r'(\w+):\s*"([^"]+)"', param_text)
        for key, value in matches:
            params[key] = value
        return params

    def generate_with_llm(self, prompt, params):
        """ Gửi prompt đến Gemini để sinh nội dung """
        model = genai.GenerativeModel("gemini-2.0-flash")
        full_prompt = f"{prompt}\n\n{params}" if params else prompt
        response = model.generate_content(full_prompt)
        return response.text.strip() if response and response.text else "[LLM Error]"

    def export_notebook(self, filename="lesson.ipynb"):
        """ Xuất nội dung thành file Jupyter Notebook """
        nb = nbformat.v4.new_notebook()

        # Thêm tiêu đề bài học
        nb.cells.append(nbformat.v4.new_markdown_cell(f"# {self.lesson['title']}\n"))

        # Thêm từng phần nội dung
        for section in self.lesson["sections"]:
            nb.cells.append(nbformat.v4.new_markdown_cell(f"## {section['title']}\n"))
            nb.cells.append(nbformat.v4.new_markdown_cell(section["content"]))

        # Ghi ra file
        with open(filename, "w", encoding="utf-8") as f:
            nbformat.write(nb, f)
        print(f"Notebook đã được lưu: {filename}")

# Ví dụ DSL
dsl_code = """
lesson "Lập trình Python cơ bản" {
    section "Giới thiệu" {
        content "Python là một ngôn ngữ lập trình bậc cao, dễ học và mạnh mẽ."
    }
    section "Cấu trúc điều kiện" {
        generate "Giải thích về câu lệnh if-else trong Python"
    }
    section "Vòng lặp" {
        generate "Giới thiệu về vòng lặp for và while trong Python" {
            parameters {
                length: "short"
                examples: "2"
            }
        }
    }
}
"""

# Xử lý DSL và tạo Notebook
processor = LessonDSLProcessor(dsl_code)
processor.parse()
processor.export_notebook("D:/python_lesson.ipynb")


Notebook đã được lưu: D:/python_lesson.ipynb
